# Auth Service Testing (Simplified)

This notebook tests if the AuthService is working correctly with our PostgreSQL client.

In [ ]:
import os
import sys
import json
from pathlib import Path
from dotenv import load_dotenv

# Add backend directory to path to import our modules
notebook_dir = Path(os.getcwd())
backend_dir = notebook_dir / 'backend'
sys.path.append(str(backend_dir))

# Load environment variables from .env file
load_dotenv(dotenv_path=".env")
import pandas as pd
import uuid

In [ ]:
from app.db.pg_client import test_connection, execute_query

# Verify we can connect to the database
if test_connection():
    print("✅ Database connection successful!")
else:
    print("❌ Database connection failed!")

# Check if users table exists and examine its schema
schema_query = """
SELECT 
    column_name, 
    data_type, 
    is_nullable, 
    column_default
FROM 
    information_schema.columns
WHERE 
    table_schema = 'public'
    AND table_name = 'users'
ORDER BY 
    ordinal_position;
"""

try:
    columns = execute_query(schema_query)
    print("\nUsers table schema:")
    display(pd.DataFrame(columns))
except Exception as e:
    print(f"\n❌ Error querying users table schema: {e}")

In [ ]:
# Test creating a user directly with SQL
def create_test_user():
    try:
        # Generate a unique email and user ID
        test_user_id = str(uuid.uuid4())
        test_email = f"test_{test_user_id[:8]}@example.com"
        
        # Use a sample password hash (this would normally be properly hashed)
        password_hash = "sample_hash_for_testing_purposes_only"
        
        # Execute the insert
        query = """
        INSERT INTO users (id, email, password_hash, role, created_at)
        VALUES (%s, %s, %s, %s, NOW())
        RETURNING id, email, role, created_at
        """
        
        result = execute_query(query, (test_user_id, test_email, password_hash, "user"), fetchall=False)
        
        print(f"✅ Successfully created test user with email: {test_email}")
        display(pd.DataFrame([result]))
        
        return result
    except Exception as e:
        print(f"❌ Error creating test user: {e}")
        return None
    
# Create a test user
test_user = create_test_user()

In [ ]:
# Now test our AuthService directly
from app.services.auth_service import AuthService

# Test user registration with AuthService
def test_auth_service_signup():
    try:
        test_email = f"auth_test_{uuid.uuid4().hex[:8]}@example.com"
        test_password = "TestPassword123!"
        
        print(f"Attempting to register user: {test_email}")
        result, error = AuthService.sign_up(test_email, test_password, {"role": "user"})
        
        if error:
            print(f"❌ Error during registration: {error}")
            return None
        else:
            print("✅ Successfully registered user:")
            print(f"User ID: {result['user']['id']}")
            print(f"Email: {result['user']['email']}")
            print(f"Access token: {result['session']['access_token'][:20]}...")
            return result
    except Exception as e:
        print(f"❌ Exception during registration: {e}")
        return None

# Test the sign-up function
signup_result = test_auth_service_signup()

In [ ]:
# Test user sign-in with AuthService
def test_auth_service_signin(email, password):
    try:
        print(f"Attempting to sign in user: {email}")
        result, error = AuthService.sign_in(email, password)
        
        if error:
            print(f"❌ Error during sign-in: {error}")
            return None
        else:
            print("✅ Successfully signed in user:")
            print(f"User ID: {result['user']['id']}")
            print(f"Email: {result['user']['email']}")
            print(f"Access token: {result['session']['access_token'][:20]}...")
            return result
    except Exception as e:
        print(f"❌ Exception during sign-in: {e}")
        return None

# Test sign-in with the user we just created
if signup_result:
    signin_result = test_auth_service_signin(signup_result['user']['email'], "TestPassword123!")

In [ ]:
# Verify we can query and view all users
try:
    users = execute_query("SELECT id, email, role, created_at FROM users ORDER BY created_at DESC LIMIT 10")
    print(f"Found {len(users)} users in the database:")
    display(pd.DataFrame(users))
except Exception as e:
    print(f"❌ Error querying users: {e}")